# Loading Necessary Libraries

In [4]:
import pandas as pd, numpy as np, matplotlib.pyplot as plt, csv
# These are standard python modules
import json, time, urllib.parse
# The 'requests' module is not a standard Python module. You will need to install this with pip/pip3 if you do not already have it
import requests

In [5]:
politicians = pd.read_csv('../politicians_by_country_AUG.2024.csv')
population  = pd.read_csv('../population_by_country_AUG.2024.csv')

In [25]:
politicians.head()

,name,url,country
0,Majah Ha Adrif,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,Afghanistan
1,Haroon al-Afghani,https://en.wikipedia.org/wiki/Haroon_al-Afghani,Afghanistan
2,Tayyab Agha,https://en.wikipedia.org/wiki/Tayyab_Agha,Afghanistan
3,Khadija Zahra Ahmadi,https://en.wikipedia.org/wiki/Khadija_Zahra_Ah...,Afghanistan
4,Aziza Ahmadyar,https://en.wikipedia.org/wiki/Aziza_Ahmadyar,Afghanistan


In [26]:
population.head()

,Geography,Population
0,WORLD,8009.0
1,AFRICA,1453.0
2,NORTHERN AFRICA,256.0
3,Algeria,46.8
4,Egypt,105.2


In [27]:
#########
#
#    CONSTANTS
#

# The basic English Wikipedia API endpoint
API_ENWIKIPEDIA_ENDPOINT = "https://en.wikipedia.org/w/api.php"
API_HEADER_AGENT = 'User-Agent'

# We'll assume that there needs to be some throttling for these requests - we should always be nice to a free data resource
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = (1.0/100.0)-API_LATENCY_ASSUMED

# When making automated requests we should include something that is unique to the person making the request
# This should include an email - your UW email would be good to put in there
REQUEST_HEADERS = {
    'User-Agent': 'igokhale@uw.edu, University of Washington, MSDS DATA 512 - AUTUMN 2024'
}

# This is just a list of English Wikipedia article titles that we can use for example requests
ARTICLE_TITLES = [ 'Bison', 'Northern flicker', 'Red squirrel', 'Chinook salmon', 'Horseshoe bat' ]

# This is a string of additional page properties that can be returned see the Info documentation for
# what can be included. If you don't want any this can simply be the empty string
PAGEINFO_EXTENDED_PROPERTIES = "talkid|url|watched|watchers"
#PAGEINFO_EXTENDED_PROPERTIES = ""

# This template lists the basic parameters for making this
PAGEINFO_PARAMS_TEMPLATE = {
    "action": "query",
    "format": "json",
    "titles": "",           # to simplify this should be a single page title at a time
    "prop": "info",
    "inprop": PAGEINFO_EXTENDED_PROPERTIES
}

In [28]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_pageinfo_per_article(article_title = None, 
                                 endpoint_url = API_ENWIKIPEDIA_ENDPOINT, 
                                 request_template = PAGEINFO_PARAMS_TEMPLATE,
                                 headers = REQUEST_HEADERS):
    
    # article title can be as a parameter to the call or in the request_template
    if article_title:
        request_template['titles'] = article_title

    if not request_template['titles']:
        raise Exception("Must supply an article title to make a pageinfo request.")

    if API_HEADER_AGENT not in headers:
        raise Exception(f"The header data should include a '{API_HEADER_AGENT}' field that contains your UW email address.")

    if 'uwnetid@uw' in headers[API_HEADER_AGENT]:
        raise Exception(f"Use your UW email address in the '{API_HEADER_AGENT}' field.")

    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free
        # data source like Wikipedia - or any other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        response = requests.get(endpoint_url, headers=headers, params=request_template)
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [29]:
#########
#
#    CONSTANTS
#

#    The current LiftWing ORES API endpoint and prediction model
#
API_ORES_LIFTWING_ENDPOINT = "https://api.wikimedia.org/service/lw/inference/v1/models/{model_name}:predict"
API_ORES_EN_QUALITY_MODEL = "enwiki-articlequality"

#
#    The throttling rate is a function of the Access token that you are granted when you request the token. The constants
#    come from dissecting the token and getting the rate limits from the granted token. An example of that is below.
#
API_LATENCY_ASSUMED = 0.002       # Assuming roughly 2ms latency on the API and network
API_THROTTLE_WAIT = ((60.0*60.0)/5000.0)-API_LATENCY_ASSUMED  # The key authorizes 5000 requests per hour

#    When making automated requests we should include something that is unique to the person making the request
#    This should include an email - your UW email would be good to put in there
#    
#    Because all LiftWing API requests require some form of authentication, you need to provide your access token
#    as part of the header too
#
REQUEST_HEADER_TEMPLATE = {
    'User-Agent': "<{email_address}>, University of Washington, MSDS DATA 512 - AUTUMN 2024",
    'Content-Type': 'application/json',
    'Authorization': "Bearer {access_token}"
}
#
#    This is a template for the parameters that we need to supply in the headers of an API request
#
REQUEST_HEADER_PARAMS_TEMPLATE = {
    'email_address' : "igokhale@uw.edu",         # your email address should go here
    'access_token'  : ""          # the access token you create will need to go here
}

#
#    A dictionary of English Wikipedia article titles (keys) and sample revision IDs that can be used for this ORES scoring example
#
ARTICLE_REVISIONS = { 'Bison':1085687913 , 'Northern flicker':1086582504 , 'Red squirrel':1083787665 , 'Chinook salmon':1085406228 , 'Horseshoe bat':1060601936 }

#
#    This is a template of the data required as a payload when making a scoring request of the ORES model
#
ORES_REQUEST_DATA_TEMPLATE = {
    "lang":        "en",     # required that its english - we're scoring English Wikipedia revisions
    "rev_id":      "",       # this request requires a revision id
    "features":    True
}

#
#    These are used later - defined here so they, at least, have empty values
#
USERNAME = ""
ACCESS_TOKEN = ""
#

In [30]:
#from apikeys.KeyManager import KeyManager
#keyman = KeyManager()

#
#   This is my Wikipedia/Wikimedia username. They suggest you request your keys using your Wikipedia username, so I
#   also stored the API key using my Wikipedia username.
#
#   You should probably use your own username here.
#USERNAME = ""
#key_info = keyman.findRecord(USERNAME,API_ORES_LIFTWING_ENDPOINT)
#ACCESS_TOKEN = key_info[0]['key']
#print(key_info[0]['description'])
#print(ACCESS_TOKEN)
#
#   Note: if you don't want to use the key manager to help manage your API keys, you can specify the values as constants
#   below. Just don't distribute the notebook without removing the constants or you'll be distributing your key too.
#
#

In [32]:
#########
#
#    PROCEDURES/FUNCTIONS
#

def request_ores_score_per_article(article_revid = None, email_address=None, access_token=None,
                                   endpoint_url = API_ORES_LIFTWING_ENDPOINT, 
                                   model_name = API_ORES_EN_QUALITY_MODEL, 
                                   request_data = ORES_REQUEST_DATA_TEMPLATE, 
                                   header_format = REQUEST_HEADER_TEMPLATE, 
                                   header_params = REQUEST_HEADER_PARAMS_TEMPLATE):
    
    #    Make sure we have an article revision id, email and token
    #    This approach prioritizes the parameters passed in when making the call
    if article_revid:
        request_data['rev_id'] = article_revid
    if email_address:
        header_params['email_address'] = email_address
    if access_token:
        header_params['access_token'] = access_token
    
    #   Making a request requires a revision id - an email address - and the access token
    if not request_data['rev_id']:
        raise Exception("Must provide an article revision id (rev_id) to score articles")
    if not header_params['email_address']:
        raise Exception("Must provide an 'email_address' value")
    if not header_params['access_token']:
        raise Exception("Must provide an 'access_token' value")
    
    # Create the request URL with the specified model parameter - default is a article quality score request
    request_url = endpoint_url.format(model_name=model_name)
    
    # Create a compliant request header from the template and the supplied parameters
    headers = dict()
    for key in header_format.keys():
        headers[str(key)] = header_format[key].format(**header_params)
    
    # make the request
    try:
        # we'll wait first, to make sure we don't exceed the limit in the situation where an exception
        # occurs during the request processing - throttling is always a good practice with a free data
        # source like ORES - or other community sources
        if API_THROTTLE_WAIT > 0.0:
            time.sleep(API_THROTTLE_WAIT)
        #response = requests.get(request_url, headers=headers)
        response = requests.post(request_url, headers=headers, data=json.dumps(request_data))
        json_response = response.json()
    except Exception as e:
        print(e)
        json_response = None
    return json_response

In [ ]:
def get_final_csv():
    politicians = pd.read_csv('Downloads/politicians_by_country_AUG.2024.csv')
    pred = {}
    for article in politicians.name.to_list():
        info = request_pageinfo_per_article(article)
        page_id = list(info["query"]["pages"].keys())[0]
        if "lastrevid" not in list(info["query"]["pages"][page_id].keys()):
            print("could not find lastrevid for", article)
            continue
        rev_id = info["query"]["pages"][page_id]["lastrevid"]
        score = request_ores_score_per_article(article_revid= rev_id,
                                        email_address="igokhale@uw.edu",
                                        access_token=ACCESS_TOKEN)
        if score is None:
            continue
        if 'enwiki' not in list(score.keys()):
            print('nothing in score from this', article)
            continue
        rev_id = str(rev_id)
        if rev_id not in list(score['enwiki']['scores'].keys()):
            print("could not find rev id")
            continue
        quality = score['enwiki']['scores'][rev_id]['articlequality']['score']['prediction']
        pred[article] = quality

    with open('quality_pred.csv', mode='w', newline='') as file:
        print('writing csv')
        writer = csv.writer(file)
        
        # Write the header
        writer.writerow(['article', 'quality_pred'])
        
        # Write the data
        for key, value in pred.items():
            writer.writerow([key, value])

```
could not find lastrevid for Barbara Eibinger-Miedl  
could not find lastrevid for Mehrali Gasimov  
nothing in score from this Presidency of Eurico Gaspar Dutra  
could not find lastrevid for Kyaw Myint  
could not find lastrevid for André Ngongang Ouandji  
nothing in score from this Wang Wei (politician, born 1974)  
could not find lastrevid for Tomás Pimentel  
could not find lastrevid for Richard Sumah  
nothing in score from this Ajoy Edwards  
nothing in score from this Anup Dhotre  
nothing in score from this Viola Carofalo  
nothing in score from this Ferdinando Palasciano  
nothing in score from this Kim Ok-gyun  
nothing in score from this Pierre Ramampy  
could not find lastrevid for Segun ''Aeroland'' Adewale  
nothing in score from this Anwar Ali Hyder  
nothing in score from this Nancy Steele  
could not find lastrevid for Bashir Bililiqo  
nothing in score from this Abdul-Aziz al-Khair  
nothing in score from this Chada Thaiseth  
nothing in score from this Radhia Haddad
```

In [62]:
21/7090

0.0029619181946403386

In [51]:
def get_rev_id():
    revision = {}
    for article in politicians.name.to_list():
        info = request_pageinfo_per_article(article)
        page_id = list(info["query"]["pages"].keys())[0]
        if "lastrevid" not in list(info["query"]["pages"][page_id].keys()):
            print("could not find lastrevid for", article)
            continue
        rev_id = info["query"]["pages"][page_id]["lastrevid"]
        revision[article] = rev_id
    with open('rev_id.csv', mode='w', newline='') as file:
        print('writing csv')
        writer = csv.writer(file)
            
        # Write the header
        writer.writerow(['article', 'rev_id'])
            
        # Write the data
        for key, value in revision.items():
            writer.writerow([key, value])

could not find lastrevid for Barbara Eibinger-Miedl
could not find lastrevid for Mehrali Gasimov
could not find lastrevid for Kyaw Myint
could not find lastrevid for André Ngongang Ouandji
could not find lastrevid for Tomás Pimentel
could not find lastrevid for Richard Sumah
could not find lastrevid for Segun ''Aeroland'' Adewale
could not find lastrevid for Bashir Bililiqo


In [6]:
rev_id_df = pd.read_csv('rev_id.csv')
pred = pd.read_csv('../../quality_pred.csv')

In [7]:
display(politicians.head())
population.head()

,name,url,country
0,Majah Ha Adrif,https://en.wikipedia.org/wiki/Majah_Ha_Adrif,Afghanistan
1,Haroon al-Afghani,https://en.wikipedia.org/wiki/Haroon_al-Afghani,Afghanistan
2,Tayyab Agha,https://en.wikipedia.org/wiki/Tayyab_Agha,Afghanistan
3,Khadija Zahra Ahmadi,https://en.wikipedia.org/wiki/Khadija_Zahra_Ah...,Afghanistan
4,Aziza Ahmadyar,https://en.wikipedia.org/wiki/Aziza_Ahmadyar,Afghanistan


,Geography,Population
0,WORLD,8009.0
1,AFRICA,1453.0
2,NORTHERN AFRICA,256.0
3,Algeria,46.8
4,Egypt,105.2


In [8]:
data = pd.merge(rev_id_df, pred, on = 'article')
data.head()

,article,rev_id,quality_pred
0,Majah Ha Adrif,1233202991,Start
1,Haroon al-Afghani,1230459615,B
2,Tayyab Agha,1225661708,Start
3,Khadija Zahra Ahmadi,1234741562,Stub
4,Aziza Ahmadyar,1195651393,Start


In [9]:
pol_df = pd.merge(politicians, data, left_on = 'name', right_on = 'article')[['country', 'article', 'rev_id' , 'quality_pred']]
pol_df.head()

,country,article,rev_id,quality_pred
0,Afghanistan,Majah Ha Adrif,1233202991,Start
1,Afghanistan,Haroon al-Afghani,1230459615,B
2,Afghanistan,Tayyab Agha,1225661708,Start
3,Afghanistan,Khadija Zahra Ahmadi,1234741562,Stub
4,Afghanistan,Aziza Ahmadyar,1195651393,Start


In [10]:
regions = []
countries = []

# Current region tracker
current_region = None

# Iterate through each entry
for entry in population.Geography:
    if entry.isupper():  # Check if the entry is a region
        current_region = entry  # Update current region
    else:
        regions.append(current_region)  # Assign current region to the country
        countries.append(entry)  # Add country to the countries list

# Create new DataFrame with regions and countries
new_pop = pd.DataFrame({
    'Country': countries,
    'Region': regions
})

In [11]:
pop_df = pd.merge(new_pop, population, left_on = 'Country', right_on = 'Geography')[['Country', 'Region', 'Population']]

In [12]:
pop_df.head()

,Country,Region,Population
0,Algeria,NORTHERN AFRICA,46.8
1,Egypt,NORTHERN AFRICA,105.2
2,Libya,NORTHERN AFRICA,6.9
3,Morocco,NORTHERN AFRICA,37.0
4,Sudan,NORTHERN AFRICA,48.1


In [13]:
final_df = pd.merge(pol_df, pop_df, left_on = 'country', 
                    right_on = 'Country')[['country', 'Region', 'Population','article', 'rev_id', 
                    'quality_pred']].rename(columns={'Region': 'region',
                                                     'Population': 'population', 
                                                     'article': 'article_title',
                                                     'rev_id': 'revision_id',
                                                     'quality_pred': 'article_quality'})


In [14]:
final_df.head()

,country,region,population,article_title,revision_id,article_quality
0,Afghanistan,SOUTH ASIA,42.4,Majah Ha Adrif,1233202991,Start
1,Afghanistan,SOUTH ASIA,42.4,Haroon al-Afghani,1230459615,B
2,Afghanistan,SOUTH ASIA,42.4,Tayyab Agha,1225661708,Start
3,Afghanistan,SOUTH ASIA,42.4,Khadija Zahra Ahmadi,1234741562,Stub
4,Afghanistan,SOUTH ASIA,42.4,Aziza Ahmadyar,1195651393,Start


In [15]:
final_df.to_csv('./wp_politicians_by_country.csv')

In [16]:
no_match = pd.merge(pol_df, pop_df, left_on = 'country', 
                    right_on = 'Country', how='outer', indicator=True)

# Identify countries with no matches
no_match_df = no_match[no_match['_merge'] != 'both']

# Get lists of non-matching countries
no_match_countries = no_match_df['country'].dropna().unique().tolist()

# Save the non-matching countries to a text file
with open('./wp_countries-no_match.txt', 'w') as f:
    for country in no_match_countries:
        f.write(f"{country}\n")

Your analysis will consist of calculating total-articles-per-capita (a ratio representing the number of articles per person)  and high-quality-articles-per-capita (a ratio representing the number of high quality articles per person) on a country-by-country and regional basis.

Top 10 countries by coverage: The 10 countries with the highest total articles per capita (in descending order) .
Bottom 10 countries by coverage: The 10 countries with the lowest total articles per capita (in ascending order) .
Top 10 countries by high quality: The 10 countries with the highest high quality articles per capita (in descending order) .
Bottom 10 countries by high quality: The 10 countries with the lowest high quality articles per capita (in ascending order).
Geographic regions by total coverage: A rank ordered list of geographic regions (in descending order) by total articles per capita.
Geographic regions by high quality coverage: Rank ordered list of geographic regions (in descending order) by high quality articles per capita.


In [18]:
final_df.head()

,country,region,population,article_title,revision_id,article_quality
0,Afghanistan,SOUTH ASIA,42.4,Majah Ha Adrif,1233202991,Start
1,Afghanistan,SOUTH ASIA,42.4,Haroon al-Afghani,1230459615,B
2,Afghanistan,SOUTH ASIA,42.4,Tayyab Agha,1225661708,Start
3,Afghanistan,SOUTH ASIA,42.4,Khadija Zahra Ahmadi,1234741562,Stub
4,Afghanistan,SOUTH ASIA,42.4,Aziza Ahmadyar,1195651393,Start


### Articles per Capita (County)

In [47]:
num_articles = final_df.groupby('country')['article_title'].count().reset_index(name='num_articles')

pop_data = final_df[['country', 'population']].drop_duplicates()

df2 = pd.merge(num_articles, pop_data, on='country')

df2['total-articles-per-capita'] = df2['num_articles'] /df2['population']

In [48]:
df2.sort_values(by = 'total-articles-per-capita', ascending = False)[2:12]

,country,num_articles,population,total-articles-per-capita
4,Antigua and Barbuda,33,0.1,330.000000
51,Federated States of Micronesia,14,0.1,140.000000
93,Marshall Islands,13,0.1,130.000000
149,Tonga,10,0.1,100.000000
12,Barbados,25,0.3,83.333333
98,Montenegro,36,0.6,60.000000
125,Seychelles,6,0.1,60.000000
90,Maldives,33,0.6,55.000000
17,Bhutan,44,0.8,55.000000
121,Samoa,8,0.2,40.000000


In [49]:
df2.sort_values(by = 'total-articles-per-capita', ascending = True)[:10]

,country,num_articles,population,total-articles-per-capita
31,China,15,1411.3,0.010628
57,Ghana,3,34.1,0.087977
66,India,149,1428.6,0.104298
122,Saudi Arabia,5,36.9,0.135501
164,Zambia,3,20.2,0.148515
108,Norway,1,5.5,0.181818
70,Israel,2,9.8,0.204082
45,Egypt,32,105.2,0.304183
37,Cote d'Ivoire,10,30.9,0.323625
50,Ethiopia,44,126.5,0.347826


### Articles per Capita (Region)

In [50]:
num_articles = final_df.groupby('region')['article_title'].count().reset_index(name='num_articles')

regions_df = population[population['Geography'].str.isupper()].drop_duplicates()

df2 = pd.merge(num_articles, regions_df, left_on='region', right_on = 'Geography')

df2['total-articles-per-capita_region'] = df2['num_articles'] /df2['Population']

In [51]:
df2.sort_values(by = 'total-articles-per-capita_region', ascending = False)[:10]

,region,num_articles,Geography,Population,total-articles-per-capita_region
14,SOUTHERN EUROPE,795,SOUTHERN EUROPE,152.0,5.230263
0,CARIBBEAN,218,CARIBBEAN,44.0,4.954545
17,WESTERN EUROPE,497,WESTERN EUROPE,199.0,2.497487
5,EASTERN EUROPE,709,EASTERN EUROPE,285.0,2.487719
16,WESTERN ASIA,608,WESTERN ASIA,299.0,2.033445
8,NORTHERN EUROPE,191,NORTHERN EUROPE,108.0,1.768519
13,SOUTHERN AFRICA,123,SOUTHERN AFRICA,70.0,1.757143
9,OCEANIA,72,OCEANIA,45.0,1.600000
4,EASTERN AFRICA,663,EASTERN AFRICA,483.0,1.372671
10,SOUTH AMERICA,568,SOUTH AMERICA,426.0,1.333333


In [52]:
df2.sort_values(by = 'total-articles-per-capita_region', ascending = True)[:10]

,region,num_articles,Geography,Population,total-articles-per-capita_region
3,EAST ASIA,151,EAST ASIA,1648.0,0.091626
11,SOUTH ASIA,667,SOUTH ASIA,2029.0,0.328733
12,SOUTHEAST ASIA,394,SOUTHEAST ASIA,682.0,0.577713
1,CENTRAL AMERICA,188,CENTRAL AMERICA,182.0,1.032967
6,MIDDLE AFRICA,230,MIDDLE AFRICA,202.0,1.138614
15,WESTERN AFRICA,512,WESTERN AFRICA,442.0,1.158371
7,NORTHERN AFRICA,301,NORTHERN AFRICA,256.0,1.175781
2,CENTRAL ASIA,106,CENTRAL ASIA,80.0,1.325000
10,SOUTH AMERICA,568,SOUTH AMERICA,426.0,1.333333
4,EASTERN AFRICA,663,EASTERN AFRICA,483.0,1.372671


### High Quality Articles per Capita (Country)

In [53]:
high_quality = final_df[(final_df['article_quality'] == 'FA') | (final_df['article_quality'] == 'GA')].groupby('country')['article_title'].count().reset_index(name='num_high_quality')

pop_data = final_df[['country', 'population']].drop_duplicates()

quality = pd.merge(high_quality, pop_data, on='country')

quality['high-quality-articles-per-capita'] = quality['num_high_quality'] /quality['population']

In [54]:
quality.sort_values(by = 'high-quality-articles-per-capita', ascending = False)[:10]

,country,num_high_quality,population,high-quality-articles-per-capita
63,Montenegro,3,0.6,5.000000
56,Luxembourg,2,0.7,2.857143
1,Albania,7,2.7,2.592593
50,Kosovo,4,1.7,2.352941
58,Maldives,1,0.6,1.666667
55,Lithuania,4,2.9,1.379310
25,Croatia,5,3.8,1.315789
40,Guyana,1,0.8,1.250000
70,Palestinian Territory,6,5.5,1.090909
81,Slovenia,2,2.1,0.952381


In [55]:
quality.sort_values(by = 'high-quality-articles-per-capita', ascending = True)[:10]

,country,num_high_quality,population,high-quality-articles-per-capita
9,Bangladesh,1,173.5,0.005764
29,Egypt,1,105.2,0.009506
31,Ethiopia,2,126.5,0.015810
46,Japan,2,124.5,0.016064
69,Pakistan,4,240.5,0.016632
22,Colombia,1,52.2,0.019157
23,Congo DR,2,102.3,0.019550
100,Vietnam,2,98.9,0.020222
95,Uganda,1,48.6,0.020576
2,Algeria,1,46.8,0.021368


### High Quality Articles per Capita (Region)

In [56]:
high_quality = final_df[(final_df['article_quality'] == 'FA') | (final_df['article_quality'] == 'GA')].groupby('region')['article_title'].count().reset_index(name='num_high_quality')

regions_df = population[population['Geography'].str.isupper()].drop_duplicates()

df2 = pd.merge(high_quality, regions_df, left_on='region', right_on = 'Geography')

df2['high-quality-articles-per-capita_region'] = df2['num_high_quality'] / df2['Population']

In [58]:
df2.sort_values(by = 'high-quality-articles-per-capita_region', ascending = False)[:10]

,region,num_high_quality,Geography,Population,high-quality-articles-per-capita_region
14,SOUTHERN EUROPE,53,SOUTHERN EUROPE,152.0,0.348684
0,CARIBBEAN,9,CARIBBEAN,44.0,0.204545
5,EASTERN EUROPE,38,EASTERN EUROPE,285.0,0.133333
13,SOUTHERN AFRICA,8,SOUTHERN AFRICA,70.0,0.114286
17,WESTERN EUROPE,21,WESTERN EUROPE,199.0,0.105528
16,WESTERN ASIA,27,WESTERN ASIA,299.0,0.090301
8,NORTHERN EUROPE,9,NORTHERN EUROPE,108.0,0.083333
7,NORTHERN AFRICA,17,NORTHERN AFRICA,256.0,0.066406
2,CENTRAL ASIA,5,CENTRAL ASIA,80.0,0.062500
1,CENTRAL AMERICA,10,CENTRAL AMERICA,182.0,0.054945
